<a href="https://colab.research.google.com/github/AySay02/Tweets-Classification/blob/main/08_Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP and model fundamentals


In [ ]:
import pandas as pd
import random
import tensorflow as tf

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2228992d-9905-c945-a662-fe261e8e3bb6)


In [ ]:
# Getting Helper functions

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data , create_tensorboard_callback , plot_loss_curves , compare_historys

--2023-06-02 07:09:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-02 07:09:17 (104 MB/s) - ‘helper_functions.py’ saved [10246/10246]



**Getting text dataset**\
Kaggle's basic NLP dataset ; disaster classification

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-02 07:09:18--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 64.233.170.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   718KB/s    in 0.8s    

2023-06-02 07:09:19 (718 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
# Visualizing a text Dataset

# To visualize the text samples , we need to read them

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Shuffling dataset

train_df_shuffled = train_df.sample(frac=1 , random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


No target ie whether disaster or not column is present in test dataset for obv reason

In [ ]:
len(train_df) , len(test_df)

(7613, 3263)

In [ ]:
# Visualizing random samples
random_index = random.randint(0,len(train_df) - 5)
#create random indexes not higher than the length

for row in train_df_shuffled[["text" , "target"]][random_index:random_index+5].itertuples():
  _ , text , target = row
  print(f"Target :{target}" , "(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("-----\n")

Target :0 (not real disaster)
Text:
#PBBan (Temporary:300) fighterdena @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason

-----

Target :1 (real disaster)
Text:
@TheSmallClark 'He'll kill me instead if he survived the shot. I don't exactly know. I fled the scene and pulled the trigger with---

-----

Target :0 (not real disaster)
Text:
One thing you can be sure of. There will never be bush fires in Scotland as the ground is always soaking wet????

-----

Target :0 (not real disaster)
Text:
@Trollkrattos Juan Carlos Salvador The Secret Tips to Get 100.000 Riot Points LoL are out now! check the Secret on on my Bio

-----

Target :0 (not real disaster)
Text:
Ignition Knock (Detonation) Sensor-Senso Standard fits 02-06 Acura RSX 2.0L-L4 http://t.co/VZaIQAMDCp http://t.co/ycecN44c8P

-----



# Splitting data sets

In [ ]:
# split data into training and validation sets

from sklearn.model_selection import train_test_split


In [ ]:
# splitting

train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                               train_df_shuffled["target"].to_numpy(),
                                                                               test_size=0.1,# 10% of data for validation
                                                                               random_state=42)

In [ ]:
train_sentences[:5] , train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object),
 array([0, 0, 1, 0, 0]))

# ***TOKENIZATION AND EMBEDDING***
**Tokenization** - straight mapping from token to number(gets too big)\
**Embedding**- create a matrix of feature vector of tokens(can be learned)

`Text Vectorization (tokenization)`

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# this is a DEFAULT TextVect. param

text_vectorizer = TextVectorization(max_tokens=None,# how many words in vocabulary;None means no limit
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,# create groups of n words
                                    output_mode="int",#how to map tokens into num
                                    output_sequence_length=None# how long you want your sequences to be
                                    )

In [ ]:
# Finding the average number of tokens (words) in trainset

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# setting up tex vect. vars

max_vocab_length = 10000  # max nums of words in our vocab
max_length = 15   # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)



```
# Fit the text vectorizer to training set
```



In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
# create a sample sentences and tokenize it

sample_sent = "There's a flood here!"
text_vectorizer([sample_sent])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232, 140,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# getting the vocab ; most common words and least common

word_in_vocab = text_vectorizer.get_vocabulary()
top_5 = word_in_vocab[:5]
least_5 = word_in_vocab[-5:]

print(f"Top 5 words : {top_5}")
print(f"Least 5 words : {least_5}")

Top 5 words : ['', '[UNK]', 'the', 'a', 'in']
Least 5 words : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


**Create an Embedding layer**

In [ ]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length , #input shape
                      output_dim = 128, # output shape
                      input_length = max_length # how long each input
                      )
embedding

In [ ]:
# get a random sentence from trainset
rand_sent = random.choice(train_sentences)

print(f"Original sentence : {rand_sent}\
        \n\nEmbedding Version : ")

# embed the random sentence (turn text into dense vector of fixed size)

sample_embed = embedding(text_vectorizer([rand_sent])) # first into number conversion is important
sample_embed

Original sentence : Israel wrecked my home. Now it wants my land. 
https://t.co/g0r3ZR1nQj        

Embedding Version : 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02386436, -0.04764616, -0.0281808 , ...,  0.02162858,
          0.02647335,  0.03772653],
        [-0.01653626,  0.04817483, -0.02101632, ...,  0.03784559,
          0.00095423,  0.04317922],
        [ 0.02891326,  0.04493472,  0.02089853, ...,  0.00099596,
          0.00646358,  0.01160705],
        ...,
        [-0.02743645, -0.02670559,  0.01818975, ..., -0.02607914,
         -0.02389035,  0.00322197],
        [-0.02743645, -0.02670559,  0.01818975, ..., -0.02607914,
         -0.02389035,  0.00322197],
        [-0.02743645, -0.02670559,  0.01818975, ..., -0.02607914,
         -0.02389035,  0.00322197]]], dtype=float32)>

In [ ]:
# check out a single token's embedding

sample_embed[0][0] , sample_embed[0][0].shape , rand_sent[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00098925,  0.04653994, -0.02439513,  0.02153168,  0.01127124,
         0.01609306, -0.0135648 , -0.03849079, -0.03995124, -0.04473505,
         0.0315646 , -0.00928897,  0.01359913,  0.02098859, -0.00543288,
        -0.02396668, -0.04515962,  0.01608906, -0.02405332,  0.04433027,
         0.02717086, -0.01314848, -0.04848145,  0.02239174, -0.01870972,
         0.0345982 , -0.03503584,  0.0458903 ,  0.04582569,  0.0139333 ,
        -0.01620799,  0.03294451,  0.01755339, -0.01206533,  0.04993606,
        -0.01897625, -0.00279555,  0.0334827 ,  0.01599715,  0.01437238,
        -0.00206081,  0.03076044, -0.04533581,  0.03655523,  0.03117817,
        -0.01971423,  0.01863886,  0.04034492, -0.00402304, -0.02723709,
        -0.00250192,  0.04963082,  0.0021091 ,  0.01666192, -0.03388538,
         0.01348719,  0.01551557, -0.04477176, -0.01037661,  0.00542253,
        -0.0121279 ,  0.01956587,  0.0450444 ,  0.02999339,  0.00012437,
  

# Modelling datasets

`Baseline model : Naive Bayes text classifier`- from SKlearn ml map

In [ ]:
# Model 0  Naive bayes multinomial using TF-IDF formula
# pretrained model from scikit learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ("tfidf" , TfidfVectorizer()),    # convert words into number using tfidf
    ("clf" , MultinomialNB())   # model the text
])

model_0.fit(train_sentences , train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# evaluate our baseline model

baseline_score = model_0.score(val_sentences , val_labels)
print(f"Baseline accuracy : {baseline_score*100:.2f}%")

Baseline accuracy : 79.27%


In [ ]:
# making preds

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
# creating an evaluation function for model experiments(Binary classification)
# metrics used: Accuracy , precision , Recall, F1-score(comb of pres and recall)

from sklearn.metrics import accuracy_score , precision_recall_fscore_support

def calculate_results(y_true , y_pred):
  model_accuracy = accuracy_score(y_true , y_pred)*100
  # calculate model pres , recall , f1 score using "weighted" average
  model_precision , model_recall , model_f1 , _ = precision_recall_fscore_support(y_true , y_pred , average = "weighted")
  model_results = {"accuracy" : model_accuracy,
                   "precison" : model_precision,
                   "recall": model_recall,
                   "F1 score" : model_f1}
  return model_results

In [ ]:
# getting baseline res

baseline_res = calculate_results(y_true = val_labels,
                                 y_pred = baseline_preds)
baseline_res

{'accuracy': 79.26509186351706,
 'precison': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1 score': 0.7862189758049549}

`Model-1: a simple dense model; Feed Forward NN`

In [ ]:
# Creating a tensorboard callback

from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [ ]:
from tensorflow.keras import layers

#Using Functional API

inputs = layers.Input(shape=(1,) , dtype=tf.string) # inputs are 1-D strings
x = text_vectorizer(inputs) # turn input text into numbers

x = embedding(x)  # Turned into an embedding

x = layers.GlobalAveragePooling1D()(x)  # condense feature vect for each token to one vector

outputs = layers.Dense(1 , activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs , outputs , name="Model_1_Dense")


In [ ]:
model_1.summary()

Model: "Model_1_Dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model1_hist = model_1.fit(x=train_sentences,
            y=train_labels,
            epochs=5 ,
            validation_data=(val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                   experiment_name="model_1_dense")])


Saving TensorBoard log files to: model_logs/model_1_dense/20230531-190847
Epoch 1/5
215/215 [==============================] - 20s 70ms/step - loss: 0.6117 - accuracy: 0.6929 - val_loss: 0.5350 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4419 - accuracy: 0.8184 - val_loss: 0.4723 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3475 - accuracy: 0.8609 - val_loss: 0.4586 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2841 - accuracy: 0.8898 - val_loss: 0.4634 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2371 - accuracy: 0.9127 - val_loss: 0.4836 - val_accuracy: 0.7822


In [ ]:
model_1.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4836 - accuracy: 0.7822


[0.4835765063762665, 0.7821522355079651]

In [ ]:
model1_pred_prob = model_1.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [ ]:
model1_pred_prob[0]

array([0.30878842], dtype=float32)

In [ ]:
model_1_preds = tf.squeeze(tf.round(model1_pred_prob))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [ ]:
# Calculate model1 results

model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precison': 0.7860403042839984,
 'recall': 0.7821522309711286,
 'F1 score': 0.779380517102918}

In [ ]:
baseline_res

{'accuracy': 79.26509186351706,
 'precison': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1 score': 0.7862189758049549}

# Visualizing learned embeddings

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
# Getting the weight matrix in embedding layer

embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights

[array([[-0.04215103, -0.03934054, -0.029697  , ..., -0.00525402,
          0.0155007 , -0.0430063 ],
        [ 0.05213743, -0.016587  , -0.03198849, ...,  0.01848623,
          0.00176709, -0.05731197],
        [-0.01746763,  0.02868311,  0.05110298, ..., -0.01349006,
          0.05183384, -0.03373195],
        ...,
        [ 0.04493565, -0.00861676, -0.04579037, ...,  0.02943773,
         -0.01567906, -0.01077133],
        [-0.00306269,  0.00237057,  0.018502  , ..., -0.02486364,
          0.07076647, -0.03876638],
        [ 0.08466585, -0.03970369,  0.07189727, ..., -0.03021527,
          0.04809141, -0.02408769]], dtype=float32)]

In [ ]:
embed_weights[0].shape

(10000, 128)

To visualize the embedding matrix ; tensorflow has a got a tool called **Projector**

In [ ]:
# Create Embedding files (from tensorflow word embedding documentation)

# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0:
#      continue # skip 0 , padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()


In [ ]:
# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

# Recurrent Neural Network(RNNs)
>Useful for Sequence data.\
>Basically learns from previous representation and applies them to current representation

In [ ]:
# creating an LSTM model

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,) , dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units = 64 , return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(1 , activation = "sigmoid")(x)

outputs = layers.Dense(1 , activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs , outputs , name="model_2_LSTM")

when stacking the RNN layers ; you should return the sequences as the layers change the dim of the input shapes

In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

In [ ]:
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model2_hist = model_2.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230531-192354
Epoch 1/5
215/215 [==============================] - 20s 57ms/step - loss: 0.4707 - accuracy: 0.8933 - val_loss: 0.5708 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4146 - accuracy: 0.9399 - val_loss: 0.5626 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3800 - accuracy: 0.9502 - val_loss: 0.5669 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3522 - accuracy: 0.9518 - val_loss: 0.5599 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3250 - accuracy: 0.9574 - val_loss: 0.5587 - val_accuracy: 0.7651


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[0.15556273],
       [0.6734258 ],
       [0.67342734],
       [0.1545793 ],
       [0.15450045],
       [0.6734232 ],
       [0.67334956],
       [0.67342716],
       [0.6734268 ],
       [0.15594627]], dtype=float32)

In [ ]:
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# calculate res

model_2_res = calculate_results(val_labels,
                                model_2_pred)
model_2_res

{'accuracy': 76.50918635170603,
 'precison': 0.7664434345240916,
 'recall': 0.7650918635170604,
 'F1 score': 0.7630272521222509}

In [ ]:
baseline_res

{'accuracy': 79.26509186351706,
 'precison': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1 score': 0.7862189758049549}

# GRU POWERED RNN

Gated recurrent unit; very effective RNN componenet; very similar but less parameters

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# for stacking LSTM layers; return sequences = true

# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)

# x = layers.Dense(64 , activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x)  # this can reduce the shape to 1 token

outputs = layers.Dense(1 , activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs , outputs , name="Model_3")


In [ ]:
model_3.summary()

Model: "Model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________

In [ ]:
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model3_hist = model_3.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230531-194451
Epoch 1/5
215/215 [==============================] - 14s 52ms/step - loss: 0.2059 - accuracy: 0.9267 - val_loss: 0.6065 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1302 - accuracy: 0.9610 - val_loss: 0.7162 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1118 - accuracy: 0.9651 - val_loss: 0.6738 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0981 - accuracy: 0.9675 - val_loss: 0.7282 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0883 - accuracy: 0.9685 - val_loss: 0.7219 - val_accuracy: 0.7782


In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)

24/24 [==============================] - 1s 4ms/step


In [ ]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_3_res = calculate_results(val_labels,
                                model_3_pred)
model_3_res

{'accuracy': 77.82152230971128,
 'precison': 0.7796479981380524,
 'recall': 0.7782152230971129,
 'F1 score': 0.7763875633348853}

# Bidirectional RNN
Basically consists of two RNNs that processes left to right and other right to left\
Helps the model to learn from both forward and backward hence giving accurate predictions

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,) , dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

# Bidirectional() is a wrapper class ; any RNN layer can be inserted

x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)

outputs = layers.Dense(1 , activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs , outputs , name="Model_4_Bi")

In [ ]:
model_4.summary()

Model: "Model_4_Bi"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-

In [ ]:
model_4.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model4_hist =  model_4.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_4_Bi")])

Saving TensorBoard log files to: model_logs/model_4_Bi/20230602-072155
Epoch 1/5
215/215 [==============================] - 25s 73ms/step - loss: 0.5061 - accuracy: 0.7514 - val_loss: 0.4584 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.3092 - accuracy: 0.8748 - val_loss: 0.4691 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2060 - accuracy: 0.9254 - val_loss: 0.5574 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1376 - accuracy: 0.9507 - val_loss: 0.7069 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0938 - accuracy: 0.9660 - val_loss: 0.8971 - val_accuracy: 0.7598


In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)


24/24 [==============================] - 1s 3ms/step


In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [ ]:
model_4_res = calculate_results(val_labels,
                                model_4_preds)
model_4_res

{'accuracy': 75.98425196850394,
 'precison': 0.7633887291688025,
 'recall': 0.7598425196850394,
 'F1 score': 0.7565467702363975}

In [ ]:
baseline_res

{'accuracy': 79.26509186351706,
 'precison': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1 score': 0.7862189758049549}

# 1D Convulational NN
Basically for text and other types of sequence data

In [ ]:
# Basic embedding test for conv1d layer

from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
# this turns target into embedding

conv_1d = layers.Conv1D(filters=64,
                       kernel_size=5,   # like ngram=5 ie looks at 5 words at a time
                       activation="relu", # strides = 1 by deafault
                       padding="same") # by default valid ; changes the shape if valid(padding)
                        # same padding basically covers all the words
conv_1d_output = conv_1d(embedding_test)  # pass the embedding through the layer
max_pool = layers.GlobalMaxPool1D()
# instead of 32 per token, it pools to 32 per sequence ie whole sentence
max_pool_output = max_pool(conv_1d_output)  # equivalent to getting the most important feature

embedding_test.shape , conv_1d_output.shape , max_pool_output.shape


(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [ ]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00156729, -0.0208911 , -0.00221805, ..., -0.00899066,
          0.04576629, -0.0080641 ],
        [ 0.04415649, -0.01869665, -0.04056321, ..., -0.00760169,
          0.03993336, -0.02327592],
        [-0.04086774, -0.0394332 , -0.0235105 , ...,  0.0086579 ,
          0.00749658,  0.04509317],
        ...,
        [-0.03037559, -0.02199673,  0.00706886, ..., -0.01409803,
         -0.00185908,  0.00864911],
        [-0.03037559, -0.02199673,  0.00706886, ..., -0.01409803,
         -0.00185908,  0.00864911],
        [-0.03037559, -0.02199673,  0.00706886, ..., -0.01409803,
         -0.00185908,  0.00864911]]], dtype=float32)>

In [ ]:
conv_1d_output

<tf.Tensor: shape=(1, 15, 64), dtype=float32, numpy=
array([[[0.00000000e+00, 4.59350273e-02, 0.00000000e+00, 1.34038469e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.30633132e-02,
         0.00000000e+00, 4.31243628e-02, 7.53372209e-03, 3.77796069e-02,
         0.00000000e+00, 3.79593335e-02, 0.00000000e+00, 4.67623724e-03,
         0.00000000e+00, 1.32408496e-02, 1.14246532e-02, 0.00000000e+00,
         8.84696282e-03, 2.69203428e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.89266310e-03,
         1.40145710e-02, 1.76322646e-02, 0.00000000e+00, 0.00000000e+00,
         3.49655445e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 1.04831578e-02, 2.74942107e-02, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 4.41999435e-02, 0.00000000e+00,
         1.46374833e-02, 1.77132320e-02, 4.08485644e-02, 0.00000000e+00,
         0.00000000e+00, 6.09189563e-04, 0.00000000e+00, 0.00000000e+00

In [ ]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.03420688, 0.04593503, 0.03875961, 0.04701634, 0.02758761,
        0.06899696, 0.04990816, 0.03333473, 0.01163819, 0.04929606,
        0.03591813, 0.04184256, 0.00795801, 0.03795933, 0.04618482,
        0.03571614, 0.06564758, 0.05623969, 0.03838843, 0.05502545,
        0.05887093, 0.03337291, 0.02653291, 0.02812728, 0.04975544,
        0.03816232, 0.01302036, 0.06826787, 0.04864368, 0.0530409 ,
        0.0570797 , 0.05217439, 0.0685861 , 0.08427295, 0.01628065,
        0.07728556, 0.02458656, 0.04729555, 0.05602574, 0.0364832 ,
        0.02638274, 0.03615872, 0.04419994, 0.03162135, 0.04457485,
        0.04606874, 0.04084856, 0.05901059, 0.07522458, 0.04264278,
        0.02437258, 0.09289413, 0.03598963, 0.03983035, 0.01437721,
        0.02675603, 0.02200456, 0.03825188, 0.03125741, 0.06432468,
        0.00509746, 0.06481817, 0.02662401, 0.05063385]], dtype=float32)>

In [ ]:
# building a conv1d model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  activation="relu",
                  strides=1,
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1 , activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs,outputs, name="model_5_conv1d")

In [ ]:
model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_5 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 1)              

In [ ]:
model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model5_hist =  model_5.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_5_conv1d")])

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230602-075707
Epoch 1/5
215/215 [==============================] - 14s 50ms/step - loss: 0.2452 - accuracy: 0.9240 - val_loss: 0.6144 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1288 - accuracy: 0.9543 - val_loss: 0.7339 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0868 - accuracy: 0.9723 - val_loss: 0.8171 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0718 - accuracy: 0.9749 - val_loss: 0.8775 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0642 - accuracy: 0.9781 - val_loss: 0.8883 - val_accuracy: 0.7651


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))


In [ ]:
model_5_res = calculate_results(val_labels,
                                model_5_preds)
model_5_res

{'accuracy': 76.50918635170603,
 'precison': 0.765325307490926,
 'recall': 0.7650918635170604,
 'F1 score': 0.7637474572934756}

# Transfer Learning
We'll select an embedding (encoder) from tensorflow hub

In [ ]:
import tensorflow_hub as hub
sample_sentence = "There's a flood in my street!"
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embed_samples = embed([sample_sentence,
                       "well this is a sample sentence for the universal encoder"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [ ]:
embed_samples[0].shape

TensorShape([512])

In [ ]:
#Create a keras layer using the USE pretrained layer from tensorflowhub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],   # as it could be a variable length;read doc of this
                                        dtype=tf.string,
                                        trainable=False,
                                        name = "USE")

In [ ]:
# create the model using sequential API

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64 , activation="relu"),
    layers.Dense(1 , activation="sigmoid")
] , name="model_6_USE")

model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_3 (Dense)             (None, 64)                32832     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
# train a classifier on top of USE
model6_hist = model_6.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_6_USE")])

Saving TensorBoard log files to: model_logs/model_6_USE/20230602-082453
Epoch 1/5
215/215 [==============================] - 6s 14ms/step - loss: 0.5053 - accuracy: 0.7811 - val_loss: 0.4505 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4149 - accuracy: 0.8154 - val_loss: 0.4357 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4004 - accuracy: 0.8213 - val_loss: 0.4292 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3927 - accuracy: 0.8301 - val_loss: 0.4288 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3861 - accuracy: 0.8308 - val_loss: 0.4308 - val_accuracy: 0.8097


In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_res = calculate_results(val_labels , model_6_preds)
model_6_res

24/24 [==============================] - 1s 14ms/step


{'accuracy': 80.97112860892388,
 'precison': 0.8132253018714792,
 'recall': 0.8097112860892388,
 'F1 score': 0.8077085649633902}

In [ ]:
baseline_res

{'accuracy': 79.26509186351706,
 'precison': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1 score': 0.7862189758049549}

# With only 10% of data

In [ ]:
# train_10_percent = train_df_shuffled[["text" , "target"]].sample(frac=0.1,random_state=42)

In [ ]:
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()


In [ ]:
# train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

To avoid data leakage we split from train_sentences as repeted sentences will give higher accuracy

In [ ]:
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]


In [ ]:
# TO recreate a model same as a model previously created we use;

model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# train a classifier on top of USE
model7_hist = model_7.fit(train_sentences_10_percent,
                          train_labels_10_percent,
                          epochs=5,
                          validation_data = (val_sentences , val_labels),
                          callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                 "model_7_USE_10_percent")])


Saving TensorBoard log files to: model_logs/model_7_USE_10_percent/20230602-090246
Epoch 1/5
22/22 [==============================] - 4s 46ms/step - loss: 0.6749 - accuracy: 0.6920 - val_loss: 0.6524 - val_accuracy: 0.7362
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 0.6078 - accuracy: 0.8058 - val_loss: 0.5923 - val_accuracy: 0.7664
Epoch 3/5
22/22 [==============================] - 1s 24ms/step - loss: 0.5306 - accuracy: 0.8073 - val_loss: 0.5359 - val_accuracy: 0.7677
Epoch 4/5
22/22 [==============================] - 1s 28ms/step - loss: 0.4668 - accuracy: 0.8204 - val_loss: 0.5046 - val_accuracy: 0.7638
Epoch 5/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4238 - accuracy: 0.8263 - val_loss: 0.4880 - val_accuracy: 0.7690


In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_res = calculate_results(val_labels , model_7_preds)
model_7_res

24/24 [==============================] - 1s 10ms/step


{'accuracy': 76.9028871391076,
 'precison': 0.7693974434525301,
 'recall': 0.7690288713910761,
 'F1 score': 0.7676527036976099}

# Results comparing in github of tutorial

In [ ]:
# upload to tensorboard.dev

!tensorboard dev upload --logdir ./model_logs/ \
  --name "NLP modelling experiments Course" \
  --description "Comparing different types of models" \
  --one_shot # exit the uploader once done

2023-06-02 09:14:17.290219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   NYWL-XNPB



New experiment created. View your TensorBoard at: https://tensorboard.dev/experime

In [ ]:
# to view all experiments
!tensorboard dev list

2023-06-02 09:17:29.657505: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
https://tensorboard.dev/experiment/Z6imwDNLTBOKBSjOUrGrcw/
	Name                 NLP modelling experiments Course
	Description          Comparing different types of models
	Id                   Z6imwDNLTBOKBSjOUrGrcw
	Created              2023-06-02 09:14:50 (2 minutes ago)
	Updated              2023-06-02 09:14:56 (2 minutes ago)
	Runs                 10
	Tags                 5
	Scalars              150
	Tensor bytes         0
	Binary object bytes  3340749
Total: 1 experiment(s)


In [ ]:
# to delete
# !tensorboard dev delete --<experiment_id>

# saving and uploading model in github of course